# Lectura datos

In [36]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import plotly.express as px

In [37]:
import pandas as pd
data_path = 'ml-100k/'

# Read the u.data file into a pandas DataFrame
udata = pd.read_csv(data_path+'u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
genre= pd.read_csv(data_path+'u.genre', sep='|', header=None, names=['genre', 'id'])
moviesinfo = pd.read_csv(data_path+'u.item', sep='|', header=None,encoding='latin-1', names=[ 'movie id' , 'movie title' , 'release date' , 'video release date' ,
            "IMDb UR'L" , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
            "Children's" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
            'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
            'Thriller' , 'War' , 'Western' ,])

userd = pd.read_csv(data_path+'u.user', sep='|', header=None, names=['user id' , 'age', 'gender' , 'occupation' , 'zip code'])




# Revisión de datos


In [38]:
udata.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [39]:
userd.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [40]:
genre.head()

,genre,id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [41]:
 # to remove the video release sinces it's emtpy but it's necessary to read the csv.
moviesinfo= moviesinfo.drop(columns='video release date')
moviesinfo.head()

,movie id,movie title,release date,IMDb UR'L,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Descripcion estadística

### Users

Las siguientes visuales tienen como finalidad mostrar cómo se encuentran distribuidas los elementos de cada tabla de datos, para con ello estalecer un balance en el entrenamiento y entender también cómo podrían influir en el modelo de recomendación. 

In [42]:
px.bar(userd.groupby('gender').count().reset_index(), x= 'gender', y = 'user id', color = 'gender', title=' Gender Distribution') 

In [43]:
px.bar(userd.groupby('occupation').count().reset_index().sort_values('user id', ascending=False), x= 'occupation', y = 'user id', color = 'occupation', title=' Gender Distribution')

In [44]:
px.violin(userd, x= 'age', color='gender')

### ratings

In [45]:
udata.item_id.value_counts()

item_id
50      583
258     509
100     508
181     507
294     485
       ... 
852       1
1505      1
1653      1
1452      1
1641      1
Name: count, Length: 1682, dtype: int64

In [46]:
px.bar(udata.rating.value_counts())

### Movies

In [47]:
moviesinfo.shape[0] - moviesinfo.drop_duplicates('movie title').shape[0] #Hay 18 titulos de peliculas repetidos con diferentes movie id

18

In [48]:
moviesinfo[moviesinfo['movie title']== 'Designated Mourner, The (1997)']

,movie id,movie title,release date,IMDb UR'L,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1255,1256,"Designated Mourner, The (1997)",23-May-1997,http://us.imdb.com/M/title-exact?Designated%20...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1256,1257,"Designated Mourner, The (1997)",23-May-1997,http://us.imdb.com/M/title-exact?Designated%20...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# ML


In [49]:

movieeval = udata.merge(moviesinfo, left_on='item_id', right_on='movie id')

La unión de las taals tiene como fin poder explorar las clasicaciones a detalle a cada película y establecer una conneción con el usuario. 

In [50]:
wholedb = movieeval.merge(userd, left_on='user_id', right_on='user id')



Para establecer el modelo se utilizara la mayor cantidad de información relevante posible, para ello se cruzan todas las tablas, de aquí se desecharán las columnas que no aportan valor a la hora de establecer el modelo.


In [51]:
dummiesdb = pd.concat([wholedb, pd.get_dummies(wholedb['occupation']),pd.get_dummies(wholedb.gender)], axis = 1)

In [52]:
dummiesdb = dummiesdb.drop(columns=['release date', 'occupation', 'gender','timestamp','movie id', "IMDb UR'L",'user id', 'zip code','movie title'])

In [53]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
# Crear las variables de entrenamiento para la clasificación.
x = dummiesdb.drop(columns ='item_id')
y = dummiesdb.item_id



# Creación y entrenamiento del modelo.
nn_model = KNeighborsClassifier()
nn_model.fit(x,y)

def get_recommendations(user_id, num_recs=10):
    #Utilizamos el indice en la tabla inicial para encontrar la posición del ID
    user_idx = dummiesdb.index.get_loc(user_id)
    
    # Luego encontramso las vecinos similares.
    # dado que iloc retorna una pandas series, covnierte en dataframe y se transpone para mantener el formato inicial. 
    distances, indices = nn_model.kneighbors(pd.DataFrame(dummiesdb.iloc[dummiesdb.index.get_loc(user_id)].drop('item_id')).T, n_neighbors=num_recs+1)
    # Aunque no es necesario se guardan las distancias 
    #Se Guarda el índice que contiene las peliculas recomendadas
    
    indices = indices[0][1:]
    
    #Se utiliza la base con los nomrbes de las películas para retornar las lista con las 10 recomendaciones.

    return moviesinfo[moviesinfo['movie id'].isin(dummiesdb.iloc[indices]['item_id'])]['movie title'].tolist()


In [54]:
get_recommendations(userd.iloc[np.random.randint(0,high = 943)]['user id'])

['Four Rooms (1995)',
 'Silence of the Lambs, The (1991)',
 'Monty Python and the Holy Grail (1974)',
 'Clockwork Orange, A (1971)',
 'Apt Pupil (1998)',
 'Vertigo (1958)',
 'Manchurian Candidate, The (1962)',
 'Cape Fear (1962)',
 'Gaslight (1944)',
 'Shallow Grave (1994)']

65566